In [ ]:
##NBA Salary Scraper - By Akshaj Dwivedula
##Importing Packages
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import Comment
import requests
import re
import time

In [ ]:
##Read CSV of player names previously scraped and construct list of columns to use in scraper 
players_list = pd.read_csv("playersalary.csv")
years = list(players_list.columns)

In [ ]:
##Collects Salary Information

for i in range(0,len(players_list)):
    name = players_list.iloc[i,1]
    ##the URL is standard for all players, so we can construct it given information previously pulled
    url = "https://www.basketball-reference.com/players/" + name[0] + "/" + name + ".html"
    page = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(page.content, "html.parser")
    body = soup.find(id = "all_all_salaries")
    #Some players did not have salary data. This accounts for that situation
    if body is None:
        players_list.at[i,'SalaryExists'] = 0
        next
    else:
        players_list.at[i,'SalaryExists'] = 1
        #Salary data was within a comment, so this breaks down the html comment to extract the data
        table = body.find(string=lambda string: isinstance(string,Comment))
        a = re.findall(r'(?<!\$)\d+(?!\,)(?!6ers)(?!ers)',table,re.MULTILINE)
        for j in range(0, (len(a)-4)//7):
            year = a[7*j +2] + "-" + a[7*j +3]
            played = year + "-played"
            salary = year + "-salary"
            if played in years:
                if players_list.loc[i,played] == 1:
                    players_list.loc[i,salary] = int(a[7*j +6])
                elif players_list.loc[i,played] == 0:
                    next
            else:
                next

In [ ]:
#Saves data to CSV
players_list.to_csv('playersalary2.csv', index = False)